In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
import numpy as np
import sklearn 
from tqdm import tqdm_notebook
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline
import catboost
import lightgbm
import xgboost
import json 
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
train = pd.read_csv('data/dota2_skill_train.csv', index_col='id')
test = pd.read_csv('data/dota2_skill_test.csv', index_col='id')

heroes = pd.read_csv('data/dota2_heroes.csv', index_col='hero_id')
abilities = pd.read_csv('data/dota2_abilities.csv', index_col='ability_id')
items = pd.read_csv('data/dota2_items.csv', index_col='item_id')

In [6]:
records = []
import json
i = 0
with open('data/dota2_skill_train.jsonlines') as fin:
    for line in tqdm_notebook(fin):
        record = json.loads(line)
        records.append(record)
        i+=1
        if i == 500:
            break

# damage_targets

In [3]:
%%time
train_damage_targets = []

with open('data/dota2_skill_train.jsonlines') as fin:
    for line in tqdm_notebook(fin):
        record = json.loads(line)
        
        damage_targets = record['damage_targets']
        damage_targets['id'] = record['id']
        
        train_damage_targets.append(damage_targets)
train_damage_targets = pd.DataFrame(train_damage_targets).set_index('id').fillna(0).astype('int')


CPU times: user 1min 13s, sys: 8.86 s, total: 1min 21s
Wall time: 1min 25s


In [4]:
%%time
test_damage_targets = []

with open('data/dota2_skill_test.jsonlines') as fin:
    for line in tqdm_notebook(fin):
        record = json.loads(line)
        
        damage_targets = record['damage_targets']
        damage_targets['id'] = record['id']
        
        test_damage_targets.append(damage_targets)
test_damage_targets = pd.DataFrame(test_damage_targets).set_index('id').fillna(0).astype('int')


CPU times: user 31.8 s, sys: 1.93 s, total: 33.7 s
Wall time: 36.3 s


# item_purchase_log, final_items

In [78]:
train_items_dict = {}

with open('data/dota2_skill_train.jsonlines') as fin:
    for line in tqdm_notebook(fin):
        record = json.loads(line)
        
        it = []
        for i in record['item_purchase_log']:
            it.append(i['item_id'])
            
        train_items_dict[record['id']] = it

train_items_qual_count = pd.DataFrame(index=train_items_dict.keys())

d = {}
for index, qual in zip(items.index,items.qual):
    d[index] = qual
    
for train_id in tqdm_notebook(train_items_qual_count.index):
    x = train_items_dict[train_id]
    qual_item = [d[i] for i in x]
    for q_i in ['component', 'rare', 'epic', 'consumable', 'artifact', 'common', 'secret_shop']:
        train_items_qual_count.at[train_id, 'qual_item_{}_count'.format(q_i)] = qual_item.count(q_i)

In [79]:
test_items_dict = {}

with open('data/dota2_skill_test.jsonlines') as fin:
    for line in tqdm_notebook(fin):
        record = json.loads(line)
        
        it = []
        for i in record['item_purchase_log']:
            it.append(i['item_id'])
            
        test_items_dict[record['id']] = it

test_items_qual_count = pd.DataFrame(index=test_items_dict.keys())

d = {}
for index, qual in zip(items.index,items.qual):
    d[index] = qual
    
for test_id in tqdm_notebook(test_items_qual_count.index):
    x = test_items_dict[test_id]
    qual_item = [d[i] for i in x]
    for q_i in ['component', 'rare', 'epic', 'consumable', 'artifact', 'common', 'secret_shop']:
        test_items_qual_count.at[test_id, 'qual_item_{}_count'.format(q_i)] = qual_item.count(q_i)

<hr>

In [107]:
items_cost_dict = {}
for ind, cost in zip(items.index, items.cost):
    items_cost_dict[ind] = cost
    
items_qual_dict = {}
for ind, qual in zip(items.index, items.qual):
    items_qual_dict[ind] = qual

In [85]:
train_final_items_dict = {}
with open('data/dota2_skill_train.jsonlines') as fin:
    for line in tqdm_notebook(fin):
        record = json.loads(line)
        train_final_items_dict[record['id']] = record['final_items']
        
train_final_items = pd.DataFrame(index=train_final_items_dict.keys())

for train_id in tqdm_notebook(train_final_items.index):
    
    x = train_final_items_dict[train_id]
    qual_items = [items_qual_dict[i] for i in x if i != 0]
    sum_cost_items = sum([items_cost_dict[i] for i in x if i != 0])
    
    train_final_items.at[train_id, 'final_items_sum'] = sum_cost_items
    for q_i in ['component', 'rare', 'epic', 'consumable', 'artifact', 'common', 'secret_shop']:
        train_final_items.at[train_id, 'final_items_{}'.format(q_i)] = qual_items.count(q_i)

In [144]:
test_final_items_dict = {}
with open('data/dota2_skill_test.jsonlines') as fin:
    for line in tqdm_notebook(fin):
        record = json.loads(line)
        test_final_items_dict[record['id']] = record['final_items']
        
test_final_items = pd.DataFrame(index=test_final_items_dict.keys())

for test_id in tqdm_notebook(test_final_items.index):
    
    x = test_final_items_dict[test_id]
    qual_items = [items_qual_dict[i] for i in x if i != 0]
    sum_cost_items = sum([items_cost_dict[i] for i in x if i != 0])
    
    test_final_items.at[test_id, 'final_items_sum'] = sum_cost_items
    for q_i in ['component', 'rare', 'epic', 'consumable', 'artifact', 'common', 'secret_shop']:
        test_final_items.at[test_id, 'final_items_{}'.format(q_i)] = qual_items.count(q_i)

# Series

In [147]:
%%time
train_series = []
train_id = []
import json
it = 0
with open('data/dota2_skill_train.jsonlines') as fin:
    for line in tqdm_notebook(fin):
        record = json.loads(line)
        
        p = np.array(record['series']['player_gold'])
        p_sum = p.sum()
        p_mean = p.mean()
        p_std = p.std()
        p_median = np.median(p)
        
        t = np.array(record['series']['dire_gold'])
        t_sum = t.sum()
        t_mean = t.mean()
        t_std = t.std()
        t_median = np.median(t)
        
        t1 = np.array(record['series']['radiant_gold'])
        t1_sum = t1.sum()
        t1_mean = t1.mean()
        t1_std = t1.std()
        t1_median = np.median(t1)
        
        
        delta_sum =  t_sum - p_sum
        delta_mean = t_mean - p_mean
        delta_std = t_std - p_std
        delta_median = t_median - p_median
        
        delta1_sum =  t1_sum - p_sum
        delta1_mean = t1_mean - p_mean
        delta1_std = t1_std - p_std
        delta1_median = t1_median - p_median
        
        
        train_id.append(record['id'])
        train_series.append([p_sum, p_mean, p_std, p_median, t_sum, t_mean, t_std, t_median, t1_sum, t1_mean, t1_std, t1_median, delta_sum, delta_mean, delta_std, delta_median, delta1_sum, delta1_mean, delta1_std, delta1_median])

train_series = pd.DataFrame(train_series, index=train_id)
train_series.columns = ['p_sum','p_mean','p_std','p_median','t_sum','t_mean','t_std','t_median', 't1_sum','t1_mean','t1_std','t1_median', 'delta_sum', 'delta_mean', 'delta_std', 'delta_median', 'delta1_sum', 'delta1_mean', 'delta1_std', 'delta1_median']

CPU times: user 2min 17s, sys: 5.55 s, total: 2min 22s
Wall time: 2min 22s


In [148]:
test_series = []
test_id = []
import json
it = 0
with open('data/dota2_skill_test.jsonlines') as fin:
    for line in tqdm_notebook(fin):
        record = json.loads(line)
        
        p = np.array(record['series']['player_gold'])
        p_sum = p.sum()
        p_mean = p.mean()
        p_std = p.std()
        p_median = np.median(p)
        
        t = np.array(record['series']['dire_gold'])
        t_sum = t.sum()
        t_mean = t.mean()
        t_std = t.std()
        t_median = np.median(t)
        
        t1 = np.array(record['series']['radiant_gold'])
        t1_sum = t1.sum()
        t1_mean = t1.mean()
        t1_std = t1.std()
        t1_median = np.median(t1)
        
        
        delta_sum =  t_sum - p_sum
        delta_mean = t_mean - p_mean
        delta_std = t_std - p_std
        delta_median = t_median - p_median
        
        delta1_sum =  t1_sum - p_sum
        delta1_mean = t1_mean - p_mean
        delta1_std = t1_std - p_std
        delta1_median = t1_median - p_median
        
        
        test_id.append(record['id'])
        test_series.append([p_sum, p_mean, p_std, p_median, t_sum, t_mean, t_std, t_median, t1_sum, t1_mean, t1_std, t1_median, delta_sum, delta_mean, delta_std, delta_median, delta1_sum, delta1_mean, delta1_std, delta1_median])

test_series = pd.DataFrame(test_series, index=test_id)
test_series.columns = ['p_sum','p_mean','p_std','p_median','t_sum','t_mean','t_std','t_median', 't1_sum','t1_mean','t1_std','t1_median', 'delta_sum', 'delta_mean', 'delta_std', 'delta_median', 'delta1_sum', 'delta1_mean', 'delta1_std', 'delta1_median']

# level up times

In [155]:
%%time

train_id_items = []
train_id = []
import json
it = 0
with open('data/dota2_skill_train.jsonlines') as fin:
    for line in tqdm_notebook(fin):
        record = json.loads(line)
        
        m = np.mean(record['level_up_times'])
        s = np.std(record['level_up_times'])
        med = np.median(record['level_up_times'])
        ma =np.max(record['level_up_times'])
        
        train_id_items.append([m,s,med,ma])
        train_id.append(record['id'])
        
df_train = pd.DataFrame(train_id_items, index=train_id)
df_train.columns = ['l_u_mean', 'l_u_std', 'l_u_med', 'l_u_ma']

CPU times: user 1min 26s, sys: 5.11 s, total: 1min 31s
Wall time: 1min 31s


In [157]:
test_id_items = []
test_id = []
import json
it = 0
with open('data/dota2_skill_test.jsonlines') as fin:
    for line in tqdm_notebook(fin):
        record = json.loads(line)
        
        m = np.mean(record['level_up_times'])
        s = np.std(record['level_up_times'])
        med = np.median(record['level_up_times'])
        ma =np.max(record['level_up_times'])
        
        test_id_items.append([m,s,med,ma])
        test_id.append(record['id'])

df_test = pd.DataFrame(test_id_items, index=test_id)
df_test.columns = ['l_u_mean', 'l_u_std', 'l_u_med', 'l_u_ma']

# radiant_heroes, dire_heroes, heroes

In [158]:
from sklearn.preprocessing import OneHotEncoder

In [165]:
train_teammates = []
train_id = []
import json
it = 0
with open('data/dota2_skill_train.jsonlines') as fin:
    for line in tqdm_notebook(fin):
        record = json.loads(line)
        
        train_id.append(record['id'])
        train_teammates.append(record['{}_heroes'.format(record['player_team'])])
        
train_teammates_id = pd.DataFrame(train_teammates, index=train_id)
train_teammates= pd.DataFrame(index=train_id)

h0 = OneHotEncoder().fit_transform(train_teammates_id[0].values.reshape(-1,1))
h1 = OneHotEncoder().fit_transform(train_teammates_id[1].values.reshape(-1,1))
h2 = OneHotEncoder().fit_transform(train_teammates_id[2].values.reshape(-1,1))
h3 = OneHotEncoder().fit_transform(train_teammates_id[3].values.reshape(-1,1))
h4 = OneHotEncoder().fit_transform(train_teammates_id[4].values.reshape(-1,1))

df_h0 = pd.DataFrame(h0.toarray(), index=train_id)
df_h1 = pd.DataFrame(h1.toarray(), index=train_id)
df_h2 = pd.DataFrame(h2.toarray(), index=train_id)
df_h3 = pd.DataFrame(h3.toarray(), index=train_id)
df_h4 = pd.DataFrame(h4.toarray(), index=train_id)

train_teammates = df_h0 + df_h1 + df_h2 + df_h3 + df_h4
train_teammates = train_teammates.astype('int')

In [166]:
test_teammates = []
test_id = []
import json
it = 0
with open('data/dota2_skill_test.jsonlines') as fin:
    for line in tqdm_notebook(fin):
        record = json.loads(line)
        
        test_id.append(record['id'])
        test_teammates.append(record['{}_heroes'.format(record['player_team'])])
        
test_teammates_id = pd.DataFrame(test_teammates, index=test_id)
test_teammates= pd.DataFrame(index=test_id)

h0 = OneHotEncoder().fit_transform(test_teammates_id[0].values.reshape(-1,1))
h1 = OneHotEncoder().fit_transform(test_teammates_id[1].values.reshape(-1,1))
h2 = OneHotEncoder().fit_transform(test_teammates_id[2].values.reshape(-1,1))
h3 = OneHotEncoder().fit_transform(test_teammates_id[3].values.reshape(-1,1))
h4 = OneHotEncoder().fit_transform(test_teammates_id[4].values.reshape(-1,1))

df_h0 = pd.DataFrame(h0.toarray(), index=test_id)
df_h1 = pd.DataFrame(h1.toarray(), index=test_id)
df_h2 = pd.DataFrame(h2.toarray(), index=test_id)
df_h3 = pd.DataFrame(h3.toarray(), index=test_id)
df_h4 = pd.DataFrame(h4.toarray(), index=test_id)

test_teammates = df_h0 + df_h1 + df_h2 + df_h3 + df_h4
test_teammates = test_teammates.astype('int')

<hr>

In [176]:
heroes_agi = {}
for ind, agi in zip(heroes.index, heroes.agi_gain):
    heroes_agi[ind] = agi
    
heroes_attack_range = {}
for ind, att in zip(heroes.index, heroes.attack_range):
    heroes_attack_range[ind] = att
    
heroes_attack_rate = {}
for ind, att_r in zip(heroes.index, heroes.attack_rate):
    heroes_attack_rate[ind] = att_r

In [177]:
train_id = []
train_tm_stats = []
for p_id in tqdm_notebook(train_teammates_id.index):
    tm_agi = []
    tm_attack_range = []
    tm_attack_rate = []
    tm_winrate = []
    
    for tm in train_teammates_id[train_teammates_id.index == p_id].values[0]:
        tm_agi.append(heroes_agi[tm])
        tm_attack_range.append(heroes_attack_range[tm])
        tm_attack_rate.append(heroes_attack_rate[tm])
        
        #tm_winrate.append(heroes[heroes.index == tm]['winrate'].values[0])
    train_id.append(p_id)

    tm_agi_mean = np.array(tm_agi).mean()
    tm_attack_range_mean = np.array(tm_attack_range).mean()
    tm_attack_rate_mean = np.array(tm_attack_rate).mean()
    #tm_winrate_mean = np.array(tm_winrate).mean()
    
    tm_agi_std = np.array(tm_agi).std()
    tm_attack_range_std = np.array(tm_attack_range).std()
    tm_attack_rate_std = np.array(tm_attack_rate).std()
    #tm_winrate_std = np.array(tm_winrate).std()
    
    
    train_tm_stats.append([tm_agi_mean, tm_attack_range_mean, tm_attack_rate_mean,
                        tm_agi_std, tm_attack_range_std, tm_attack_rate_std]) #   tm_winrate_mean, tm_winrate_std

# Heroes from Open Dota

In [225]:
import time

In [215]:
df = pd.DataFrame(index=heroes.index)

In [227]:
for index in tqdm_notebook(heroes.index):
    a = requests.get('https://api.opendota.com/api/benchmarks/?hero_id={}'.format(index)).json()
    for inf in a['result'].keys():
        s = pd.DataFrame(a['result'][inf])['value'].values
        df.at[index, '{}_median'.format(inf)] = np.median(s)
        df.at[index, '{}_mean'.format(inf)] = np.mean(s)
        df.at[index, '{}_std'.format(inf)] = np.std(s)
    time.sleep(10)

In [231]:
heroes_with_b = heroes.join(df)

In [232]:
heroes_with_b.to_csv('heroes_with_open_dota.csv')

In [234]:
df = pd.read_csv('heroes_with_open_dota.csv')

# Auto encoders

In [241]:
import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F

In [377]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        
        self.fc1 = nn.Linear(54, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 1)

    def forward(self, x):
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.fc3(x)
        return F.sigmoid(x)

net = Net()

In [379]:
criterion = nn.BCELoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [380]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

In [381]:
y = train.skilled
X = train
X = X.drop(['skilled', 'player_team', 'winner_team'], axis=1)
X = scaler.fit_transform(X)

In [384]:
for epoch in range(2):

    running_loss = 0.0
    
    for i, data in tqdm_notebook(enumerate(zip(X,y))):
        
        # get the inputs
        inputs, labels = data
        inputs, labels = torch.Tensor(inputs), torch.Tensor([labels])
        
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')

[1,  2000] loss: 0.624
[1,  4000] loss: 0.621
[1,  6000] loss: 0.611
[1,  8000] loss: 0.603
[1, 10000] loss: 0.601
[1, 12000] loss: 0.580
[1, 14000] loss: 0.602
[1, 16000] loss: 0.616
[1, 18000] loss: 0.603
[1, 20000] loss: 0.596
[1, 22000] loss: 0.600
[1, 24000] loss: 0.608
[1, 26000] loss: 0.591
[1, 28000] loss: 0.597
[1, 30000] loss: 0.602
[1, 32000] loss: 0.595
[1, 34000] loss: 0.588
[1, 36000] loss: 0.594
[1, 38000] loss: 0.582
[1, 40000] loss: 0.577
[1, 42000] loss: 0.607
[1, 44000] loss: 0.578
[1, 46000] loss: 0.590
[1, 48000] loss: 0.597
[1, 50000] loss: 0.588
[1, 52000] loss: 0.573
[1, 54000] loss: 0.587
[1, 56000] loss: 0.583
[1, 58000] loss: 0.587
[1, 60000] loss: 0.581
[1, 62000] loss: 0.591
[1, 64000] loss: 0.581
[1, 66000] loss: 0.606
[1, 68000] loss: 0.590
[1, 70000] loss: 0.579
[1, 72000] loss: 0.583
[1, 74000] loss: 0.582
[1, 76000] loss: 0.583
[1, 78000] loss: 0.587
[1, 80000] loss: 0.598
[1, 82000] loss: 0.570
[1, 84000] loss: 0.574
[1, 86000] loss: 0.597
[1, 88000] 

[2,  2000] loss: 0.589
[2,  4000] loss: 0.597
[2,  6000] loss: 0.587
[2,  8000] loss: 0.584
[2, 10000] loss: 0.584
[2, 12000] loss: 0.564
[2, 14000] loss: 0.593
[2, 16000] loss: 0.607
[2, 18000] loss: 0.590
[2, 20000] loss: 0.587
[2, 22000] loss: 0.592
[2, 24000] loss: 0.599
[2, 26000] loss: 0.581
[2, 28000] loss: 0.588
[2, 30000] loss: 0.596
[2, 32000] loss: 0.588
[2, 34000] loss: 0.580
[2, 36000] loss: 0.587
[2, 38000] loss: 0.576
[2, 40000] loss: 0.570
[2, 42000] loss: 0.602
[2, 44000] loss: 0.572
[2, 46000] loss: 0.585
[2, 48000] loss: 0.591
[2, 50000] loss: 0.582
[2, 52000] loss: 0.568
[2, 54000] loss: 0.583
[2, 56000] loss: 0.578
[2, 58000] loss: 0.583
[2, 60000] loss: 0.576
[2, 62000] loss: 0.589
[2, 64000] loss: 0.577
[2, 66000] loss: 0.602
[2, 68000] loss: 0.586
[2, 70000] loss: 0.576
[2, 72000] loss: 0.578
[2, 74000] loss: 0.578
[2, 76000] loss: 0.579
[2, 78000] loss: 0.584
[2, 80000] loss: 0.594
[2, 82000] loss: 0.568
[2, 84000] loss: 0.571
[2, 86000] loss: 0.595
[2, 88000] 

# Stacking

In [ ]:
import pandas as pd
import numpy as np
import sklearn 
from tqdm import tqdm_notebook
import catboost
import lightgbm
import xgboost
import eli5
from eli5.sklearn import PermutationImportance
from sklearn.model_selection import StratifiedKFold, cross_val_score, train_test_split
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.svm import LinearSVC
from sklearn.linear_model import SGDClassifier, Ridge
from sklearn.model_selection import ShuffleSplit, KFold, StratifiedKFold, GridSearchCV, RandomizedSearchCV
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.preprocessing import PolynomialFeatures, StandardScaler, MinMaxScaler
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.metrics import roc_auc_score
from hyperopt import hp, fmin, tpe, STATUS_OK, Trials
from sklearn.svm import SVC

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [ ]:
train, valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.5, random_state=42)

#### models

In [ ]:
lgbm1 =  lightgbm.LGBMClassifier(n_estimators=1500, learning_rate=0.1, max_depth=12, reg_alpha=9, min_child_weight=8, n_jobs=-1, random_state=1)
lgbm2 =  lightgbm.LGBMClassifier(n_estimators=1000, learning_rate=0.1, max_depth=6, reg_alpha=10, min_child_weight=5, n_jobs=-1, random_state=1)
xgb1 = xgboost.XGBClassifier(learning_rate=0.1, n_estimators=1000, max_depth=4, min_child_weight=6, gamma=0, subsample=0.8, colsample_bytree=0.8, reg_alpha=4, objective= 'binary:logistic', nthread=-1, scale_pos_weight=1, seed=27)
xgb2 = xgboost.XGBClassifier(learning_rate=0.1, n_estimators=1000, max_depth=8, min_child_weight=6, gamma=0, subsample=0.8, reg_alpha=9, nthread=6, seed=27)
cb = catboost.CatBoostClassifier(depth=8, n_estimators=1500, verbose=False) 
rg = Ridge(alpha=0.0001)
reg = LinearRegression()

In [ ]:
for clf in tqdm_notebook([lgbm1, lgbm2, xgb1, xgb2, cb, rg, reg]):
    clf.fit(train, y_train)

#### meta features for train meta and predict

In [ ]:
preds1 = lgbm1.predict(valid)
preds2 = lgbm2.predict(valid)
preds3 = xgb1.predict(valid)
preds4 = xgb2.predict(valid)
preds5 = cb.predict(valid)
preds6 = rg.predict(valid)
preds7 = reg.predict(valid)

In [ ]:
test_preds1 = lgbm1.predict(X_test)
test_preds2 = lgbm2.predict(X_test)
test_preds3 = xgb1.predict(X_test)
test_preds4 = xgb2.predict(X_test)
test_preds5 = cb.predict(X_test)
test_preds6 = rg.predict(X_test)
test_preds7 = reg.predict(X_test)

In [ ]:
st_preds = np.column_stack((preds1, preds2, preds3, preds4, preds5, preds6, preds7))
st_preds_test = np.column_stack((test_preds1, test_preds2, test_preds3, test_preds4, test_preds5, test_preds6, test_preds7))

#### unite meta with features

In [ ]:
df_meta_val = pd.DataFrame(st_preds, index=valid.index)
df_meta_test = pd.DataFrame(st_preds_test, index=X_test.index)

In [ ]:
meta_valid = pd.concat([valid, df_meta_val], axis=1, join_axes=[valid.index])

In [ ]:
meta_X_test = pd.concat([X_test, df_meta_test], axis=1, join_axes=[X_test.index])

#### meta model

In [ ]:
meta_model = LinearRegression()
meta_model.fit(meta_valid, y_valid)

In [ ]:
ans = meta_model.predict(meta_X_test)

In [ ]:
accuracy_score(y_test, (ans > 0.5).astype('int') )

# Bagging

In [ ]:
bags = 30
seed = 1 

bagged_prediction = np.zeros(X_test.shape[0])

for n in tqdm_notebook(range(0,bags)):
    my_model.set_params(random_state= seed + n)
    my_model.fit(X_train, y_train)
    preds = my_model.predict(X_test)
    bagged_prediction += preds

In [ ]:
bagged_prediction /= bags

In [ ]:
(bagged_prediction > 0.5).astype('int')

In [ ]:
accuracy_score(y_test, (bagged_prediction > 0.5).astype('int'))

# Premutation importance

In [ ]:
%%time
my_model = RandomForestClassifier(random_state=0).fit(X_train, y_train)
perm = PermutationImportance(clf_3, random_state=1).fit(X_test, y_test)

In [ ]:
eli5.show_weights(perm, feature_names = data_train.columns.tolist(), top=)